In [1]:
import os
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


In [2]:
print("📍 CWD:", os.getcwd())

DATA_PATH = "crypto_master_cleaned.csv"

if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"❌ File not found: {os.path.abspath(DATA_PATH)}")

df = pd.read_csv(DATA_PATH)
df["Date"] = pd.to_datetime(df["Date"])

df.head()


📍 CWD: O:\Project Coin\notebooks


,Date,open_price,high_price,low_price,close_price,volume,crypto
0,2020-12-31,0.184034,0.185749,0.176594,0.181397,1132268397,ADA
1,2021-01-01,0.181382,0.184246,0.172022,0.175350,1122218004,ADA
2,2021-01-02,0.175359,0.184253,0.169233,0.177423,1408849504,ADA
3,2021-01-03,0.177382,0.208679,0.173376,0.204995,2303857909,ADA
4,2021-01-04,0.205236,0.239661,0.194450,0.224762,3260699086,ADA


In [3]:
TARGET_COL = "close_price"
DATE_COL = "Date"
COIN_COL = "crypto"

LOOKBACK = 30        # days
FORECAST_DAYS = 30  # future days

coins = df[COIN_COL].unique()
coins


array(['ADA', 'BCH', 'BNB', 'BTC', 'DOGE', 'DOT', 'ETH', 'LTC', 'SOL',
       'XRP'], dtype=object)

In [4]:
def create_sequences(data, lookback):
    X, y = [], []
    for i in range(lookback, len(data)):
        X.append(data[i - lookback:i])
        y.append(data[i])
    return np.array(X), np.array(y)


In [5]:
lstm_rows = []

for coin in coins:
    print(f"🤖 Training LSTM for {coin}")

    coin_df = df[df[COIN_COL] == coin].sort_values(DATE_COL)
    prices = coin_df[TARGET_COL].values.reshape(-1, 1)

    if len(prices) < LOOKBACK + 10:
        print(f"⚠️ Skipping {coin} (not enough data)")
        continue

    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(prices)

    X, y = create_sequences(scaled, LOOKBACK)

    X = X.reshape((X.shape[0], X.shape[1], 1))

    model = Sequential([
        LSTM(50, return_sequences=True, input_shape=(LOOKBACK, 1)),
        Dropout(0.2),
        LSTM(50),
        Dense(1)
    ])

    model.compile(optimizer="adam", loss="mse")
    model.fit(X, y, epochs=10, batch_size=32, verbose=0)

    last_seq = scaled[-LOOKBACK:].reshape(1, LOOKBACK, 1)

    for step in range(FORECAST_DAYS):
        pred = model.predict(last_seq, verbose=0)
        pred_price = scaler.inverse_transform(pred)[0][0]

        forecast_date = coin_df[DATE_COL].max() + pd.Timedelta(days=step + 1)

        lstm_rows.append({
            "date": forecast_date,
            "crypto": coin,
            "forecast_price": pred_price,
            "model": "LSTM"
        })

        last_seq = np.append(last_seq[:, 1:, :], pred.reshape(1, 1, 1), axis=1)


🤖 Training LSTM for ADA


C:\Users\shind\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


🤖 Training LSTM for BCH


C:\Users\shind\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


🤖 Training LSTM for BNB


...

C:\Users\shind\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


🤖 Training LSTM for SOL


C:\Users\shind\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


🤖 Training LSTM for XRP


C:\Users\shind\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
# Save LSTM output in current working directory (notebooks/)
lstm_df = pd.DataFrame(lstm_rows)

lstm_df.to_csv("lstm_all_coins.csv", index=False)

print("✅ LSTM CSV saved in working directory (notebooks/)")


✅ LSTM CSV saved in working directory (notebooks/)


In [10]:
lstm_df.head()


,date,crypto,forecast_price,model
0,2022-03-07,ADA,0.956695,LSTM
1,2022-03-08,ADA,0.957389,LSTM
2,2022-03-09,ADA,0.960652,LSTM
3,2022-03-10,ADA,0.965711,LSTM
4,2022-03-11,ADA,0.972116,LSTM


In [11]:
lstm_df.tail()


,date,crypto,forecast_price,model
295,2022-04-01,XRP,0.772223,LSTM
296,2022-04-02,XRP,0.772784,LSTM
297,2022-04-03,XRP,0.773313,LSTM
298,2022-04-04,XRP,0.773799,LSTM
299,2022-04-05,XRP,0.774250,LSTM
